<a href="https://colab.research.google.com/github/aneeshoberoi/churnhw/blob/main/ChurnNoteBook_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
from collections import OrderedDict

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option("display.precision",2)

In [6]:
df=pd.read_csv('https://raw.githubusercontent.com/aneeshoberoi/churnhw/main/churn_homework%20-%20homework_orders.csv?token=AP7KM2RBFAH333CPSXM4ILC7SV6WI', sep=",")

In [15]:
raw=df.head(100)
raw.set_index('order_id')

,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code
order_id,,,,,,,,,,
47433960,iosapp,2019-03-13T08:02:07.000Z,Delivery,credit,0,16,13315067198,24597,MD,21162
47434359,iosapp,2019-03-13T08:11:38.000Z,Delivery,credit,0,25,15557129447,64392,NY,12944
47437194,partner_website,2019-03-13T09:07:31.000Z,Delivery,cash,0,43,25689365012,33699,NJ,8007
47442942,partner_website,2019-03-13T11:01:04.000Z,Delivery,credit,0,28,25421112251,19002,WI,53235
47443107,partner_website,2019-03-13T11:04:56.000Z,Delivery,credit,0,23,18529185191,34911,MA,2122
...,...,...,...,...,...,...,...,...,...,...
47571438,iosapp,2019-03-14T20:33:11.000Z,Delivery,credit,0,19,18312751742,57072,PA,19382
47574693,partner_website,2019-03-15T07:32:12.000Z,Delivery,cash,0,17,6103411793,15639,CT,6484
47575101,androidapp,2019-03-15T07:51:01.000Z,Delivery,credit,0,37,27267518882,30054,NJ,7921


In [ ]:
# assigning to working copy for indicator variables

# **Basic Statistics**

In [12]:
raw.customer.nunique()

87

In [13]:
raw.restaurant_total.sum()

2909.73

In [16]:
raw.order_id.nunique()

100

# ***Data Cleaning***

In [17]:
# assigning to working copy for indicator variables
test=raw

In [18]:
# Remove zero dollar txns
test=test[test['restaurant_total']>0]

In [ ]:
# Variable Conversion
test['date_purchased']=pd.to_datetime(test['date_purchased'])
test['dateonly_purchased']=test['date_purchased'].dt.date
test['houronly_purchased']=test['date_purchased'].dt.hour
test['datehour_purchased']=test['dateonly_purchased'].astype(str) + "-" + test['houronly_purchased'].astype(str)

test['restaurant_total']=test['restaurant_total'].astype(float)

In [20]:
# Removing Duplicate Txns - i.e. those made within the same hour at the same shop by a given customer. Keeping last txn only.
test=test.drop_duplicates(subset=['customer', 'shops_id', 'datehour_purchased'], keep='last')


In [21]:
# Checking Impact of cleanup

test.restaurant_total.sum()

2693.3500000000004

In [22]:
test.order_id.count()

90

# ***Data Prep - Indicators***

In [23]:
 #Enumerating Purchases
 test.sort_values(by=['customer','date_purchased'])
 
test = test.assign(order_enum=test.groupby('customer')['datehour_purchased'].cumcount()+1)
test['total_order']=test.groupby('customer')['order_id'].transform('count')

In [24]:
test['bin_orders']=np.where(test['total_order']<7,test['order_enum'],7)

In [25]:
# Calculating Journey Data

test['time_between_purch']=test.groupby(['customer'])['date_purchased'].diff()
test['spend_growth']=test.groupby(['customer'])['restaurant_total'].diff()


# Preparing Zip-Level Variables




In [262]:
#Assigning raw dataset to working copy for this section
shopszip=test[['postal_code', 'shops_id','customer','total_order']].set_index('postal_code')
shopszip=shopszip.drop_duplicates(subset=['customer', 'shops_id'], keep='last')

In [263]:
# calculating shops and customers per postal code
shopszip['shops_per_zip']=shopszip.groupby('postal_code').shops_id.nunique()
shopszip['cust_per_zip']=shopszip.groupby('postal_code').customer.nunique()
shopszip['shops_per_cust_per_zip']=shopszip['shops_per_zip']/shopszip['cust_per_zip']

In [264]:
# Of customers placing orders in a given zip, what # and  % have placed more than 2 orders in total?
temp=shopszip[shopszip['total_order']>2]
shopszip['zip_cust_2_more']=temp.groupby('postal_code')['customer'].count()
shopszip['zip_pct_cust_2_more']=shopszip['zip_cust_2_more']*100/shopszip['cust_per_zip']

In [242]:
shopszip[shopszip['shops_id']==27246]

,shops_id,customer,total_order,shops_per_zip,cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,shops_per_cust_per_zip
postal_code,,,,,,,,
98101,27246,6196987925,3,1,1,1,100,1


# ***Preparing Shop-Level Variables***

In [133]:
#Total customers served by each shop

In [285]:
shopscust=shopszip.set_index('shops_id')
shopscust=shopscust.drop(columns=['shops_per_zip','cust_per_zip','zip_cust_2_more', 'zip_pct_cust_2_more','shops_per_cust_per_zip'] )

# How many customers have placed at least one order at a given shop?

shopscust['cust_per_shop']=shopscust.groupby('shops_id').customer.nunique()

In [286]:
# Of customers placing orders at a given shop, what # and % have placed > 2 total orders/
t=shopscust[shopscust['total_order']>2]
shopscust['shop_cust_2_more']=t.groupby('shops_id')['customer'].count()
shopscust['shop_pct_cust_2_more']=shopscust['shop_cust_2_more']*100/shopscust['cust_per_shop']

# ***Preparing Shop-Zip Level Variables***

In [287]:
shopscust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 24597 to 51831
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customer              87 non-null     float64
 1   total_order           87 non-null     int64  
 2   cust_per_shop         87 non-null     int64  
 3   shop_cust_2_more      1 non-null      float64
 4   shop_pct_cust_2_more  1 non-null      float64
dtypes: float64(3), int64(2)
memory usage: 4.1 KB


In [288]:
shopscust_final=shopscust.reset_index()
shopscust_final=shopscust_final.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')


In [289]:
shopscust_final

,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more
shops_id,,,
24597,1,nan,nan
64392,1,nan,nan
33699,1,nan,nan
19002,1,nan,nan
34911,1,nan,nan
...,...,...,...
57072,1,nan,nan
15639,1,nan,nan
30054,1,nan,nan


In [298]:
shopszip_final=shopszip.reset_index()

In [ ]:

#shopscust_final=shopscust_final.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')


In [299]:
shopscustzip=shopszip_final.merge(shopscust_final,left_on='shops_id',right_on='shops_id') 

In [300]:
shopscustzip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 0 to 86
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   postal_code             87 non-null     int64  
 1   shops_id                87 non-null     int64  
 2   customer                87 non-null     float64
 3   total_order             87 non-null     int64  
 4   shops_per_zip           87 non-null     int64  
 5   cust_per_zip            87 non-null     int64  
 6   shops_per_cust_per_zip  87 non-null     float64
 7   zip_cust_2_more         1 non-null      float64
 8   zip_pct_cust_2_more     1 non-null      float64
 9   cust_per_shop           87 non-null     int64  
 10  shop_cust_2_more        1 non-null      float64
 11  shop_pct_cust_2_more    1 non-null      float64
dtypes: float64(6), int64(6)
memory usage: 8.8 KB


In [301]:
shopscustzip[shopscustzip['customer']==6196987925]

,postal_code,shops_id,customer,total_order,shops_per_zip,cust_per_zip,shops_per_cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more
77,98101,27246,6196987925,3,1,1,1,1,100,1,1,100


In [306]:
t=shopscustzip[shopscustzip['total_order']>2]
t.set_index('postal_code')

,shops_id,customer,total_order,shops_per_zip,cust_per_zip,shops_per_cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more,shops_zip_2_orders_more
postal_code,,,,,,,,,,,,
98101,27246,6196987925,3,1,1,1,1,100,1,1,100,nan


In [316]:
t['t1']=t.groupby('postal_code').shops_id.count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [312]:
t[t['customer']==6196987925]

,postal_code,shops_id,customer,total_order,shops_per_zip,cust_per_zip,shops_per_cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more,shops_zip_2_orders_more
77,98101,27246,6196987925,3,1,1,1,1,100,1,1,100,nan
